# Check Skeletons (CSV -> IMG)
This Notbook builds a qucik gradio interface, where one can manually check the csv data. Upload the csv and get an image with the detected mediapipe skeleton.

In [1]:
import cv2
import gradio as gr
import numpy as np
import mediapipe as mp
from pathlib import Path
from mediapipe.framework.formats.landmark_pb2 import NormalizedLandmark, NormalizedLandmarkList

c:\Users\hannes\01_Code\02_master_rci\03_Semester_PoliMi\02_nearables_lab\gym_tracking\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ——— Mediapipe setup ———
mp_pose    = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_styles  = mp.solutions.drawing_styles

def skeleton_csv_to_image(csv_path):
    # Read & parse the CSV
    with open(csv_path, "r") as f:
        lines = [line.strip() for line in f if line.strip()]
        lines = lines[1:]  # skip header line
        # skip last column if it exists
        lines = [line.rsplit(",", 1)[0] for line in lines]  # drop last column
    
    # each line has 132 comma-separated floats (33 landmarks * 4 values)
    data = np.array([list(map(float, line.split(","))) for line in lines], dtype=np.float32)
    num_frames = len(data)
    data = data.reshape(num_frames, 33, 4)  # → (num_frames, 33 landmarks, 4 values)
    
    # Get the last frame's keypoints
    last_frame_kpts = data[-1]
    
    # Build a NormalizedLandmarkList for the last frame
    lm_list = NormalizedLandmarkList(
        landmark=[
            NormalizedLandmark(x=pt[0], y=pt[1], z=pt[2], visibility=pt[3])
            for pt in last_frame_kpts
        ]
    )
    
    # Create a blank canvas
    canvas = np.zeros((480, 480, 3), dtype=np.uint8)
    
    # Draw skeleton
    mp_drawing.draw_landmarks(
        canvas,
        lm_list,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style()
    )
    
    return canvas

demo = gr.Interface(
    fn=skeleton_csv_to_image,
    inputs=gr.File(label="CSV File"),
    outputs=gr.Image(label="Skeleton Image"),
    title="Skeleton Visualization",
    description="Upload a CSV file to visualize the last frame's skeleton.",
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
